# 城市热点分析 - 批量Grid密度分析

## 📋 分析目标
- 遍历所有城市，生成grid密度数据
- 提取每个城市的top热点区域（支持top-n或top-percent模式）
- 生成汇总表用于后续分析和可视化
- 支持指定城市列表或全量分析

## 📊 数据源
- **输入表**: `clips_bbox_unified` - 城市bbox数据
- **分析表**: `bbox_overlap_analysis_results` - 重叠分析结果
- **密度表**: `city_grid_density` - 城市网格密度统计
- **输出表**: `city_hotspots` - 城市热点汇总（默认表名）

## ⚙️ 运行说明
- 本notebook基于 `batch_grid_analysis.py` 改造
- 可以逐步执行查看中间结果
- 支持参数调整和结果验证
- 不指定城市时自动分析所有城市


## 1. 环境设置和导入


In [ ]:
import sys
from pathlib import Path
from datetime import datetime
import subprocess
import time
import json

# 添加项目路径
project_root = Path().resolve().parent.parent
sys.path.insert(0, str(project_root))
sys.path.insert(0, str(project_root / "src"))

from spdatalab.dataset.bbox import LOCAL_DSN
from sqlalchemy import create_engine, text
import pandas as pd

print(f"📍 项目根目录: {project_root}")
print(f"🔗 数据库连接: {LOCAL_DSN[:50]}...")


## 2. 参数配置


In [ ]:
# 🎯 分析参数配置
CONFIG = {
    'output_table': 'city_top1_hotspots',  # 输出汇总表名
    'target_cities': None,  # 指定城市列表，None表示分析所有城市
    'max_cities': 3,     # 最多分析城市数量，None表示无限制 (测试用设为3)
    'grid_size': '0.002',   # 网格大小
    'density_threshold': '5', # 密度阈值
    'batch_rest_interval': 10  # 每N个城市休息一下
}

print("📋 当前配置:")
for key, value in CONFIG.items():
    print(f"   {key}: {value}")
    
print("\n💡 提示: 首次运行建议保持 max_cities=3 进行测试")


## 3. 数据库连接和基础查询


In [ ]:
# 建立数据库连接
engine = create_engine(LOCAL_DSN, future=True)
conn = engine.connect()

print("✅ 数据库连接成功")


## 4. 创建汇总表


In [ ]:
def create_top1_summary_table(conn, table_name):
    """创建top1汇总表"""
    print(f"📋 创建汇总表: {table_name}")
    
    # 先检查表是否存在
    check_table_sql = text(f"""
        SELECT EXISTS (
            SELECT FROM information_schema.tables 
            WHERE table_schema = 'public' 
            AND table_name = '{table_name}'
        );
    """)
    
    table_exists = conn.execute(check_table_sql).scalar()
    
    if not table_exists:
        # 创建新表
        create_sql = text(f"""
            CREATE TABLE {table_name} (
                id SERIAL PRIMARY KEY,
                city_id VARCHAR(50) NOT NULL,
                analysis_id VARCHAR(100),
                bbox_count INTEGER,
                subdataset_count INTEGER,
                scene_count INTEGER,
                total_overlap_area NUMERIC,
                grid_coords TEXT,
                analysis_time TIMESTAMP DEFAULT CURRENT_TIMESTAMP
            );
        """)
        conn.execute(create_sql)
        print(f"✅ 表 {table_name} 创建成功")
    else:
        print(f"📋 表 {table_name} 已存在，检查字段结构...")
        
        # 检查必要字段是否存在，如果不存在则添加
        required_fields = {
            'analysis_id': 'VARCHAR(100)',
            'bbox_count': 'INTEGER',
            'subdataset_count': 'INTEGER', 
            'scene_count': 'INTEGER',
            'total_overlap_area': 'NUMERIC',
            'grid_coords': 'TEXT'
        }
        
        for field_name, field_type in required_fields.items():
            check_field_sql = text(f"""
                SELECT EXISTS (
                    SELECT 1 FROM information_schema.columns 
                    WHERE table_name = '{table_name}' 
                    AND column_name = '{field_name}'
                    AND table_schema = 'public'
                );
            """)
            
            field_exists = conn.execute(check_field_sql).scalar()
            
            if not field_exists:
                add_field_sql = text(f"""
                    ALTER TABLE {table_name} 
                    ADD COLUMN {field_name} {field_type};
                """)
                conn.execute(add_field_sql)
                print(f"   ✅ 添加字段: {field_name} {field_type}")
    
    # 添加几何列（如果不存在）
    geometry_sql = text(f"""
        DO $$
        BEGIN
            IF NOT EXISTS (
                SELECT 1 FROM information_schema.columns 
                WHERE table_name = '{table_name}' 
                AND column_name = 'geometry'
                AND table_schema = 'public'
            ) THEN
                PERFORM AddGeometryColumn('public', '{table_name}', 'geometry', 4326, 'GEOMETRY', 2);
                RAISE NOTICE '几何列已添加到 {table_name} 表';
            END IF;
        END $$;
    """)
    conn.execute(geometry_sql)
    
    # 创建索引
    index_sql = text(f"""
        CREATE INDEX IF NOT EXISTS idx_{table_name}_city_id ON {table_name} (city_id);
        CREATE INDEX IF NOT EXISTS idx_{table_name}_bbox_count ON {table_name} (bbox_count);
        CREATE INDEX IF NOT EXISTS idx_{table_name}_geom ON {table_name} USING GIST (geometry);
    """)
    conn.execute(index_sql)
    
    conn.commit()
    print(f"✅ 表 {table_name} 结构检查完成")

# 创建汇总表
create_top1_summary_table(conn, CONFIG['output_table'])


## 5. 获取城市列表


In [ ]:
def get_all_cities(conn):
    """获取所有城市"""
    print(f"🔍 查找所有城市...")
    
    cities_sql = text("""
        SELECT 
            city_id,
            COUNT(*) as bbox_count,
            COUNT(*) FILTER (WHERE all_good = true) as good_bbox_count
        FROM clips_bbox_unified
        WHERE city_id IS NOT NULL 
        GROUP BY city_id
        ORDER BY COUNT(*) DESC;
    """)
    
    cities_df = pd.read_sql(cities_sql, conn)
    return cities_df

# 获取城市数据
cities_df = get_all_cities(conn)

print(f"📊 找到 {len(cities_df)} 个城市:")
print(cities_df.head(10))

if len(cities_df) > 10:
    print(f"... 还有 {len(cities_df) - 10} 个城市")

# 确定要分析的城市列表
if CONFIG['target_cities']:
    print(f"🎯 指定分析城市: {CONFIG['target_cities']}")
    cities_to_analyze = CONFIG['target_cities']
else:
    all_cities = cities_df['city_id'].tolist()
    if CONFIG['max_cities']:
        cities_to_analyze = all_cities[:CONFIG['max_cities']]
        print(f"🎯 分析前 {CONFIG['max_cities']} 个城市")
    else:
        cities_to_analyze = all_cities
        print(f"🎯 分析所有 {len(all_cities)} 个城市")

print(f"\n📋 将要分析的城市 ({len(cities_to_analyze)} 个):")
print(cities_to_analyze)
